## DBCSAN Clustering
`另一种聚类方式：将所有数字替换为0，不经过分词直接聚类`
``` python
re.sub(r'\d+(\.\d+)?', '0', text)
```

In [5]:
import pandas as pd
from utils.cluster import reassign_clusters, cluster, vectorize, tokenize,Cluster

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']

datasets = ['HealthApp']

for dataset in datasets:
    print('-' * 50)
    print(f'Clustering {dataset} dataset...')

    # load the dataset
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()

    # tokenize -> vectorize -> cluster -> reassign_clusters
    tokenized_logs = [tokenize(log) for log in logs]
    print(vectorize(tokenized_logs))
    labels, cluster_nums = cluster(vectorize(tokenized_logs))
    labels, cluster_nums = reassign_clusters(labels, cluster_nums, tokenized_logs)

    # store the logs in the cluster
    inputs = []
    for i in range(cluster_nums):
        inputs.append([-1, [], [], '']) # label, logs, indexs, ground_truth
    for i, label in enumerate(labels):
        inputs[label][0] = label
        inputs[label][1].append(logs[i])
        inputs[label][2].append(i)
        if inputs[label][3] == '':
            inputs[label][3] = df['EventTemplate'][i]

    clusters = []
    for input in inputs:
        c = Cluster(*input, remove_duplicate= True)
        clusters.append(c)
    Count = 0
    for c in clusters:    
        if len(c.indexs) > 10:
            print(c.logs[0])
            print(c.oracle_template)
            print(len(c.indexs))
            print('=' * 50)
            Count+=1
    print(f'Count : {Count}')

--------------------------------------------------
Clustering HealthApp dataset...
  (0, 100)	1.0
  (1, 97)	1.0
  (2, 16)	0.6248332566193072
  (2, 11)	0.5520794333347835
  (2, 98)	0.5520794333347835
  (3, 12)	0.8517032026021748
  (3, 105)	0.5240244790820354
  (4, 45)	0.5668201002003888
  (4, 121)	0.5825439786011101
  (4, 64)	0.5825439786011101
  (5, 72)	1.0
  (6, 125)	1.0
  (7, 100)	1.0
  (8, 156)	0.7071067811865475
  (8, 31)	0.7071067811865475
  (9, 155)	0.7071067811865475
  (9, 30)	0.7071067811865475
  (10, 0)	0.7064072898835545
  (10, 8)	0.7078055812151892
  (11, 97)	1.0
  (12, 100)	1.0
  (13, 100)	1.0
  (14, 97)	1.0
  (15, 72)	1.0
  (16, 125)	1.0
  :	:
  (1987, 105)	0.6960412725776134
  (1988, 13)	0.7180017735831414
  (1988, 105)	0.6960412725776134
  (1989, 13)	0.7180017735831414
  (1989, 105)	0.6960412725776134
  (1990, 13)	0.7180017735831414
  (1990, 105)	0.6960412725776134
  (1991, 13)	0.7180017735831414
  (1991, 105)	0.6960412725776134
  (1992, 13)	0.7180017735831414
  (1992, 1

## Show Clusters

In [26]:
print('num of clusters:', cluster_nums)
print('len of templates:', len(set(templates)))

# store the logs in the cluster
inputs = []
for i in range(cluster_nums):
    inputs.append([-1, [], [], '']) # label, logs, indexs, ground_truth
for i, label in enumerate(labels):
    inputs[label][0] = label
    inputs[label][1].append(logs[i])
    inputs[label][2].append(i)
    if inputs[label][3] == '':
        inputs[label][3] = df['EventTemplate'][i]

clusters = []
for input in inputs:
    c = Cluster(*input, remove_duplicate= True)
    clusters.append(c)

# for cluster in clusters:
#     print(f'cluster {cluster.label}: {len(cluster.logs)} logs, {len(cluster.indexs)} indexs')

# num = 27
# print('cluster:', num)
# print('length:', len(clusters[num].indexs))
# print('template:', clusters[num].oracle_template)
# print('len of set:', len(clusters[num].logs))
# print('-'*20)
# for log in clusters[num].logs:
#     print(log)
# print('='*40)

# for cluster in clusters:
#     if len(set(cluster.logs)) == 1 and not any(char.isdigit() for char in cluster.logs[0]):
#         print(f"{cluster.logs[0]}\n{cluster.oracle_template}")
#         print('='*40)

num of clusters: 75
len of templates: 75
onReceive action: android.intent.action.SCREEN_ON
onReceive action: android.intent.action.SCREEN_ON
processHandleBroadcastAction action:android.intent.action.SCREEN_ON
processHandleBroadcastAction action:android.intent.action.SCREEN_ON
flush sensor data
flush sensor data
onReceive action: android.intent.action.SCREEN_OFF
onReceive action: android.intent.action.SCREEN_OFF
processHandleBroadcastAction action:android.intent.action.TIME_TICK
processHandleBroadcastAction action:android.intent.action.TIME_TICK
getBinderPackageName packageName = com.huawei.health
getBinderPackageName packageName = com.huawei.health
needAutoSync autoSyncSwitch is open
needAutoSync autoSyncSwitch is open
initDataPrivacy the dataPrivacy switch is open, start push health data!
initDataPrivacy the dataPrivacy switch is open, start push health data!
initDataPrivacy the dataPrivacy is true
initDataPrivacy the dataPrivacy is <*>
initUserPrivacy the userPrivacy switch is open, 

## Evaluate

In [24]:
from evaluate import evaluate_all_datasets
from IPython.display import HTML

table = evaluate_all_datasets(
    'Test_10shot_supervised_5shot_32candidates', send_email=False)
HTML(table)

dataset,GA,PA,ED,N_ED
HDFS,1.0000,1.0000,0.0000,1.00000
Hadoop,0.9955,0.9745,0.7815,0.98931
Spark,0.9815,0.8085,1.7655,0.95721
Zookeeper,0.9945,0.9910,0.1030,0.99736
BGL,0.9935,0.9725,0.4250,0.99486
HPC,0.9525,0.9435,0.3945,0.99395
Thunderbird,0.9780,0.9500,0.5230,0.98576
Windows,0.9960,0.7075,17.3950,0.88048
Linux,0.8755,0.9645,0.2220,0.99325
Android,0.9785,0.7660,9.5145,0.93385


In [34]:
from evaluate import evaluate_single_dataset

evaluate_single_dataset(
    'outputs/parser/Test_10shot/BGL_2k.log_structured.csv', 'BGL')

         BGL: group Accuracy: 0.9845, Message-Level Accuracy: 0.8818, Edit Distance: 3.5548


In [2]:
import re

pattern = r'\b0[xX][0-9a-fA-F]+\b'
text = "Here are some hex numbers: 0x1a3, 0X4D2, and 0xABCDEF."

matches = re.findall(pattern, text)
print(matches)

['0x1a3', '0X4D2', '0xABCDEF']


## Check out some strings' freq in the whole logs or templates

In [6]:
import pandas as pd
from utils.cluster import tokenize
from utils.sample_byword import extract_variables
from utils.postprocess import correct_single_template
from utils.postprocess import extract_variables

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']


count_templates = []
for dataset in datasets:
    print(f"Processing {dataset} ----------------")
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    # templates = list(set(templates))
    for log, template in zip(logs, templates):
        if template not in count_templates and correct_single_template(template) != template:
            count_templates.append(template)
            print(template)

Processing BGL ----------------
Processing HDFS ----------------
Processing Linux ----------------
ROOT LOGIN ON <*>
Kernel command line: ro root=<*>=<*> rhgb quiet
Processing HealthApp ----------------
Processing OpenStack ----------------
Processing OpenSSH ----------------
Processing Proxifier ----------------
Processing HPC ----------------
Link error on broadcast tree Interconnect-<*>:<*>:<*>:<*>
Processing Zookeeper ----------------
My election bind port: <*>/<*>
Processing Mac ----------------
CCFile::captureLog Received Capture notice id: <*>, reason = AuthFail:sts:<*>_rsn:<*>
CCFile::captureLog Received Capture notice id: <*>, reason = DeauthInd:sts:<*>_rsn:<*>
CCFile::captureLog Received Capture notice id: <*>, reason = AssocFail:sts:<*>_rsn:<*>
CCFile::captureLog Received Capture notice id: <*>, reason = RoamFail:sts:<*>_rsn:<*>
Processing Hadoop ----------------
maxContainerCapability: <memory:<*>, vCores:<*>>
Processing Android ----------------
Processing Windows ---------

## sample based on entropy

In [ ]:
from utils.sample import sample_based_on_entropy

# datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
#         'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# for dataset in datasets:
#     pair = sample_based_on_entropy(dataset, 1)
#     print(pair[0][0])
dataset = 'HDFS'
pairs = sample_based_on_entropy(dataset, 1)
for pair in pairs:
    print(f"{pair[0][0]}\n{pair[0][1]}\n{'-'*20}")

## Mutation Count -- num

In [ ]:

# 4: 15 + 28 + 5 + 18 + 10
# 5: 1 + 2
# 6: 15 + 28 + 5 + 18 + 10

In [2]:
from utils.sample import nearest_k_pairs_from_log
pairs = [('authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=61-220-159-99.hinet-ip.hinet.net  user=root', 'authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>'), ('connection from 207.30.238.8 (host8.topspot.net) at Sun Jul 17 12:31:00 2005', 'connection from <*> (<*>) at <*>'), ('connection from 84.102.20.2 () at Sun Jul 24 02:38:22 2005', 'connection from <*> () at <*>'), ('authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=biblioteka.wsi.edu.pl', 'authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>'), ('check pass; user unknown', 'check pass; user unknown'), ('session opened for user cyrus by (uid=0)', 'session opened for user <*> by (uid=<*>)'), ('session closed for user cyrus', 'session closed for user <*>'), ('ALERT exited abnormally with [1]', 'ALERT exited abnormally with [<*>]'), ('Kerberos authentication failed', 'Kerberos authentication failed'), ('notify question section contains no SOA', 'notify question section contains no SOA'), ('Authentication failed from 163.27.187.39 (163.27.187.39): Software caused connection abort', 'Authentication failed from <*> (<*>): Software caused connection abort'), ('Authentication failed from 163.27.187.39 (163.27.187.39): Permission denied in replay cache code', 'Authentication failed from <*> (<*>): Permission denied in replay cache code'), ('restart.', 'restart.'), ('cupsd shutdown succeeded', 'cupsd shutdown succeeded'), ('cupsd startup succeeded', 'cupsd startup succeeded'), ("removing device node '/udev/vcsa2'", "removing device node '<*>'"), ("creating device node '/udev/vcs2'", "creating device node '<*>'"), ('getpeername (ftpd): Transport endpoint is not connected', 'getpeername (ftpd): Transport endpoint is not connected'), ('BIOS-e820: 00000000000f0000 - 0000000000100000 (reserved)', '<*>: <*> - <*> (reserved)'), ('ANONYMOUS FTP LOGIN FROM 84.102.20.2,  (anonymous)', 'ANONYMOUS FTP LOGIN FROM <*>,  (anonymous)'), ("warning: can't get client address: Connection reset by peer", "warning: can't get client address: Connection reset by peer"), ('BIOS-e820: 0000000000000000 - 00000000000a0000 (usable)', '<*>: <*> - <*> (usable)'), 
('User unknown timed out after 900 seconds at Sat Jun 18 02:23:10 2005', 'User unknown timed out after <*> seconds at <*>'), ('Received SNMP packet(s) from 67.170.148.126', 'Received SNMP packet(s) from <*>'), ('*** info [mice.c(1766)]:', '*** info [mice.c(<*>)]:'), ('imps2: Auto-detected intellimouse PS/2', '<*>: Auto-detected intellimouse <*>'), ('session opened for user root by LOGIN(uid=0)', 'session opened for user <*> by LOGIN(uid=<*>)'), ('ROOT LOGIN ON tty2', 'ROOT LOGIN ON <*>'), ("Couldn't authenticate user", "Couldn't authenticate user"), ('syslogd startup succeeded', 'syslogd startup succeeded'), ('klogd 1.4.1, log source = /proc/kmsg started.', 'klogd <*>, log source = <*> started.'), ('Linux version 2.6.5-1.358 (bhcompile@bugs.build.redhat.com) (gcc version 3.3.3 20040412 (Red Hat Linux 3.3.3-7)) #1 Sat May 8 09:04:50 EDT 2004', 'Linux version <*> (<*>) (gcc version <*> <*> (Red Hat Linux <*>)) <*> <*> <*> <*> <*> EDT <*>')]
print(nearest_k_pairs_from_log('connection from 61.74.96.178 () at Wed Jun 29 03:22:22 2005', pairs, 5))

[('connection from 84.102.20.2 () at Sun Jul 24 02:38:22 2005', 'connection from <*> () at <*>'), ('connection from 207.30.238.8 (host8.topspot.net) at Sun Jul 17 12:31:00 2005', 'connection from <*> (<*>) at <*>'), ('User unknown timed out after 900 seconds at Sat Jun 18 02:23:10 2005', 'User unknown timed out after <*> seconds at <*>'), ('Authentication failed from 163.27.187.39 (163.27.187.39): Software caused connection abort', 'Authentication failed from <*> (<*>): Software caused connection abort'), ('authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=61-220-159-99.hinet-ip.hinet.net  user=root', 'authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>')]
